In [1]:
# CurationStorefront Test Notebook

import os
import json
import time
from dotenv import load_dotenv
from web3 import Web3
from eth_account import Account
import requests

# Load environment variables
load_dotenv()

# Connect to the Base network
RPC_URL = os.getenv('RPC_URL')
w3 = Web3(Web3.HTTPProvider(RPC_URL))

# Load accounts
OWNER_PRIVATE_KEY = os.getenv('OWNER_PRIVATE_KEY')
owner_account = Account.from_key(OWNER_PRIVATE_KEY)
print(f"Owner account: {owner_account.address}")

# Optional curator account
if os.getenv('CURATOR_PRIVATE_KEY'):
    CURATOR_PRIVATE_KEY = os.getenv('CURATOR_PRIVATE_KEY')
    curator_account = Account.from_key(CURATOR_PRIVATE_KEY)
    print(f"Curator account: {curator_account.address}")
else:
    curator_account = None
    print("No curator account provided.")

# Contract addresses
CURATION_STOREFRONT_ADDRESS = "0xdFCA83ff7544Acb88B5D04A9101d58780243E0cb"
SUBGRAPH_URL = "https://api.studio.thegraph.com/query/90920/ump-affiliate-reviews/version/latest"

# Load contract ABIs from JSON files
def load_abi_from_file(filename):
    try:
        with open(filename, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Warning: ABI file {filename} not found. Please create this file.")
        return []
    except json.JSONDecodeError:
        print(f"Warning: ABI file {filename} contains invalid JSON.")
        return []

print("Loading contract ABIs...")
CURATION_ABI = load_abi_from_file('CurationStorefront_ABI.json')
STOREFRONT_ABI = load_abi_from_file('AffiliateERC1155Storefront_ABI.json')

# Initialize contract instances
curation_contract = w3.eth.contract(
    address=Web3.to_checksum_address(CURATION_STOREFRONT_ADDRESS),
    abi=CURATION_ABI
)

print("Contract version:", curation_contract.functions.VERSION().call())

# Helper Functions
def create_curation(name, description, payment_address, token_uri, account):
    """
    Create a new curation collection NFT
    
    Args:
        name: The name of the curation
        description: A description of the curation
        payment_address: Address that receives affiliate payments
        token_uri: URI for the NFT metadata
        account: Account object with private key
        
    Returns:
        Transaction receipt and curation ID
    """
    nonce = w3.eth.get_transaction_count(account.address)
    
    tx = curation_contract.functions.createCuration(
        name,
        description,
        payment_address,
        token_uri
    ).build_transaction({
        'from': account.address,
        'gas': 500000,
        'gasPrice': w3.eth.gas_price,
        'nonce': nonce,
    })
    
    signed_tx = account.sign_transaction(tx)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    print(f"Transaction sent: {tx_hash.hex()}")
    
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    # Find the CurationCreated event to extract the curation ID
    curation_id = None
    for log in tx_receipt.logs:
        # Look for the CurationCreated event
        # In a real implementation, we would decode the log properly using the event signature
        # For this test, we'll just assume the first successful transaction created our curation
        if log['topics'][0].hex() == '0xf3ed8c32d7d4d4be418509014d1ad98ce10da5d35e43973e668844b0c36b0395':
            # The curation ID is in the second topic (index 1)
            curation_id = int(log['topics'][1].hex(), 16)
            break
    
    if curation_id is None:
        # If we couldn't find the event, just guess that it's the most recent ID
        # This is only for testing purposes, and we could query it from the contract
        print("Could not extract curation ID from events. Using fallback method.")
        curation_id = int(tx_receipt.logs[0]['topics'][3].hex(), 16)  # Likely to be in the Transfer event
        
    print(f"Created curation with ID: {curation_id}")
    return tx_receipt, curation_id


# Test functions
def test_create_curation():
    """Test creating a new curation collection"""
    name = "Test Curation Collection"
    description = "A test curation collection for managing affiliate storefronts"
    payment_address = owner_account.address  # Use the owner as the payment address
    
    # IPFS URI for the NFT metadata
    token_uri = "ipfs://QmYwAPJzv5CZsnA625s3Xf2nemtYgPpHdWEz79ojWnPbdG"
    
    receipt, curation_id = create_curation(
        name,
        description,
        payment_address,
        token_uri,
        owner_account
    )
    
    # Verify the curation was created
    get_curation_details(curation_id)
    
    return curation_id

def test_curator_management(curation_id, curator_address):
    """Test adding and removing curators"""
    # Check initial status
    initial_status = check_curator_status(curation_id, curator_address)
    
    if not initial_status:
        # Add curator
        add_curator(curation_id, curator_address, owner_account)
        
        # Verify curator was added
        new_status = check_curator_status(curation_id, curator_address)
        assert new_status, "Curator was not added successfully"
    else:
        print(f"Address {curator_address} is already a curator")
    
    # Add the curator 
    add_curator(curation_id, curator_address, owner_account)
    # Remove curator for further testing
    time.sleep(5)
    remove_curator(curation_id, curator_address, owner_account)
    time.sleep(5)
    # add curator back for further testing
    add_curator(curation_id, curator_address, owner_account)
    time.sleep(10)
    
curation_id = test_create_curation()
    
test_curator_management(curation_id, curator_account.address)

Owner account: 0x9f4640d04371ff6b7886ade5323746388107723a
Curator account: 0x988B8c3AD5971f79D1a461D17430AfC5f1002EaA
Loading contract ABIs...
Contract version: 0.0.2
Transaction sent: 0x44ca5c57495986893a35b3807593840a9539bfc34028733f1c0228358ca0485b
Created curation with ID: 11


NameError: name 'get_curation_details' is not defined

In [10]:
import os
from dotenv import load_dotenv
from web3 import Web3
from eth_account import Account
from eth_account.messages import encode_defunct

load_dotenv()

# === SETUP ===

import os
import json
import time
from dotenv import load_dotenv
from web3 import Web3
from eth_account import Account
import requests

# Load environment variables
load_dotenv()

# Connect to the Base network
RPC_URL = os.getenv('RPC_URL')
w3 = Web3(Web3.HTTPProvider(RPC_URL))
CONTRACT_ADDRESS = "0x8fE3a8f648095CC105e01fBEB7B39B78b9778e59"
ABI = [
    {
        "inputs": [
            {"internalType": "bytes", "name": "signature", "type": "bytes"}
        ],
        "name": "submitSignature",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    }
]

# === 1. Load wallet ===
OWNER_PRIVATE_KEY = os.getenv("OWNER_PRIVATE_KEY")
owner_account = Account.from_key(OWNER_PRIVATE_KEY)
print(f"🔑 Wallet: {owner_account.address}")

# === 2. Construct message ===
region = 6  # Replace this with the correct one that matched the hash
message = f"6 regions, $1,000,000 in funding, totally based"
print(f"✉️ Message: {repr(message)}")

# === 3. Sign message (EIP-191) ===
message_hash = encode_defunct(text=message)
signed_message = Account.sign_message(message_hash, private_key=OWNER_PRIVATE_KEY)
signature = signed_message.signature
print(f"🖊️ Signature: 0x{signature.hex()}")

# === 4. Set up Web3 ===
w3 = Web3(Web3.HTTPProvider(RPC_URL))
contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=ABI)

# === 5. Build transaction ===
nonce = w3.eth.get_transaction_count(owner_account.address)
gas_price = w3.eth.gas_price
tx = contract.functions.submitSignature(signature).build_transaction({
    'from': owner_account.address,
    'nonce': nonce,
    'gas': 150000,
    'gasPrice': 741685,
    'chainId': 8453,  # Mainnet. Change to 5 for Goerli or 11155111 for Sepolia
})

# === 6. Sign and send transaction ===
signed_tx = w3.eth.account.sign_transaction(tx, private_key=OWNER_PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)

print(f"🚀 Submitted! Tx Hash: https://etherscan.io/tx/{tx_hash.hex()}")


🔑 Wallet: 0x9f4640d04371ff6b7886ade5323746388107723a
✉️ Message: '6 regions, $1,000,000 in funding, totally based'
🖊️ Signature: 0x0x0742f392355b60da34690718bac9f18a41629e72656067cf826fb889c48819812083a0d591ea7cc9faa9629ecf83a4e9dd31e9674a8dd937cda6086ef94619511c
🚀 Submitted! Tx Hash: https://etherscan.io/tx/0x3d489f85a0b373bebbbe34d79c8c6bd029f1f9958300810dd01b3ca18fc962b6


/tmp/ipykernel_330616/4292000083.py:71: DeprecationWarning: The attribute rawTransaction on SignedTransaction is deprecated in favor of raw_transaction
  tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)


In [17]:
ABI = [
    {"inputs":[],"stateMutability":"nonpayable","type":"constructor"},
    {"inputs":[],"name":"ECDSAInvalidSignature","type":"error"},
    {"inputs":[{"internalType":"uint256","name":"length","type":"uint256"}],"name":"ECDSAInvalidSignatureLength","type":"error"},
    {"inputs":[{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"ECDSAInvalidSignatureS","type":"error"},
    {"inputs":[{"internalType":"address","name":"owner","type":"address"}],"name":"OwnableInvalidOwner","type":"error"},
    {"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"OwnableUnauthorizedAccount","type":"error"},
    {"inputs":[],"name":"ReentrancyGuardReentrantCall","type":"error"},
    {"anonymous":False,"inputs":[{"indexed":True,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":True,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipTransferred","type":"event"},
    {"anonymous":False,"inputs":[{"indexed":True,"internalType":"address","name":"winner","type":"address"},{"indexed":False,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"RewardPaid","type":"event"},
    {"anonymous":False,"inputs":[{"indexed":True,"internalType":"address","name":"submitter","type":"address"},{"indexed":False,"internalType":"bool","name":"valid","type":"bool"}],"name":"SignatureSubmitted","type":"event"},
    {"stateMutability":"payable","type":"fallback"},
    {"inputs":[],"name":"CORRECT_MESSAGE_HASH","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},
    {"inputs":[],"name":"REWARD_AMOUNT","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},
    {"inputs":[{"internalType":"address","name":"_to","type":"address"},{"internalType":"bytes","name":"_data","type":"bytes"},{"internalType":"uint256","name":"_value","type":"uint256"}],"name":"escape","outputs":[],"stateMutability":"nonpayable","type":"function"},
    {"inputs":[{"internalType":"address","name":"user","type":"address"}],"name":"hasAlreadyWon","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},
    {"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},
    {"inputs":[],"name":"renounceOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},
    {"inputs":[{"internalType":"bytes","name":"signature","type":"bytes"}],"name":"submitSignature","outputs":[],"stateMutability":"nonpayable","type":"function"},
    {"inputs":[{"internalType":"address","name":"newOwner","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},
    {"inputs":[],"name":"withdraw","outputs":[],"stateMutability":"nonpayable","type":"function"},
    {"stateMutability":"payable","type":"receive"}
]


In [18]:
# === Load account ===
OWNER_PRIVATE_KEY = os.getenv("OWNER_PRIVATE_KEY")
account = Account.from_key(OWNER_PRIVATE_KEY)
print("👤 From:", account.address)

# === Build & sign message ===
message = "6 regions, $1,000,000 in funding, totally based"
eth_message = encode_defunct(text=message)
signed = Account.sign_message(eth_message, private_key=OWNER_PRIVATE_KEY)

# === Setup Web3 ===
w3 = Web3(Web3.HTTPProvider(RPC_URL))
contract = w3.eth.contract(address='0x8fE3a8f648095CC105e01fBEB7B39B78b9778e59', abi=ABI)

# === Build transaction ===
nonce = w3.eth.get_transaction_count(account.address)
gas_price = w3.eth.gas_price
tx = contract.functions.submitSignature(signed.signature).build_transaction({
    "from": account.address,
    "nonce": nonce,
    "gas": 120_000,
    "gasPrice": gas_price,
    "chainId": 8453  # Base mainnet
})
signed

👤 From: 0x9f4640d04371ff6b7886ade5323746388107723a


SignedMessage(messageHash=HexBytes('0x519c7f34d3001d14d92f887151fee1f7d12ff8bd83b7c4fec41c7ae67e47ac24'), message_hash=HexBytes('0x519c7f34d3001d14d92f887151fee1f7d12ff8bd83b7c4fec41c7ae67e47ac24'), r=3284482912491863918217634880391319278130908410085550232065209660473881401729, s=14706578157401364205119276521745653405741932452837286127316732711342598134097, v=28, signature=HexBytes('0x0742f392355b60da34690718bac9f18a41629e72656067cf826fb889c48819812083a0d591ea7cc9faa9629ecf83a4e9dd31e9674a8dd937cda6086ef94619511c'))

In [20]:
from eth_account import Account
from eth_account.messages import encode_defunct
from eth_utils import keccak

# 1. Prepare the original message
message = "6 regions, $1,000,000 in funding, totally based"
message_hash = keccak(text=message)

# 2. Apply Ethereum prefix and hash again
eth_signed_hash = keccak(b"\x19Ethereum Signed Message:\n32" + message_hash)

# 3. Sign the message
signed = Account.sign_message(encode_defunct(text=message), private_key=OWNER_PRIVATE_KEY)
signature = signed.signature
signer_address = Account.recover_message(encode_defunct(text=message), signature=signature)

# 4. Compare
print("🔐 Expected Address:", Account.from_key(OWNER_PRIVATE_KEY).address)
print("📬 Recovered Address:", signer_address)

if signer_address.lower() == Account.from_key(OWNER_PRIVATE_KEY).address.lower():
    print("✅ Signature is valid!")
else:
    print("❌ Signature verification failed!")


🔐 Expected Address: 0x9f4640d04371ff6b7886ade5323746388107723a
📬 Recovered Address: 0x9f4640d04371ff6b7886ade5323746388107723a
✅ Signature is valid!


In [13]:
from eth_account.messages import encode_defunct

message = "6 regions, $1,000,000 in funding, totally based"
eth_message = encode_defunct(text=message)
signed = Account.sign_message(eth_message, private_key=OWNER_PRIVATE_KEY)
signed


AttributeError: type object 'Account' has no attribute 'address'

In [27]:
contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=ABI)

# === Build transaction ===
nonce = w3.eth.get_transaction_count(account.address)
gas_price = w3.eth.gas_price
tx = contract.functions.submitSignature(signed.signature).build_transaction({
    "from": account.address,
    "nonce": nonce,
    "gas": 240_000,
    "gasPrice": 997265,
    "chainId": 8453  # Base mainnet
})

# === Sign and send ===
signed_tx = w3.eth.account.sign_transaction(tx, OWNER_PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
print(f"🚀 Sent: https://basescan.org/tx/{tx_hash.hex()}")

🚀 Sent: https://basescan.org/tx/0x332aeca1423e7d6c0b25150e4e85ee43031b86f1719b99d981f0eab117dbb5d1


/tmp/ipykernel_330616/2566143769.py:16: DeprecationWarning: The attribute rawTransaction on SignedTransaction is deprecated in favor of raw_transaction
  tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)


In [31]:

# === Load signer ===
PRIVATE_KEY = os.getenv("OWNER_PRIVATE_KEY")
account = Account.from_key(PRIVATE_KEY)
address = account.address
print(f"🔑 Signing from: {address}")

# === Set up Web3 and contract ===
w3 = Web3(Web3.HTTPProvider(RPC_URL))
contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=ABI)

# === Check if user already won ===
already_won = contract.functions.hasAlreadyWon(address).call()
print(f"🎯 Already won? {already_won}")
if already_won:
    exit("🚫 You've already claimed the reward.")

# === Prepare the message ===
message = "6 regions, $1,000,000 in funding, totally based"
message_hash = keccak(text=message)
eth_signed_msg = encode_defunct(hexstr=message_hash.hex())
signed = account.sign_message(eth_signed_msg)
signature = signed.signature

# === Build the transaction ===
nonce = w3.eth.get_transaction_count(address)
gas_price = w3.eth.gas_price
tx = contract.functions.submitSignature(signature).build_transaction({
    "from": address,
    "nonce": nonce,
    "gas": 320000,
    "gasPrice": 997265,
    "chainId": 8453,  # Base mainnet
})

# === Sign and send ===
signed_tx = w3.eth.account.sign_transaction(tx, PRIVATE_KEY)
tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)
print(f"🚀 Tx sent: https://basescan.org/tx/{tx_hash.hex()}")

🔑 Signing from: 0x9f4640d04371ff6b7886ade5323746388107723a
🎯 Already won? False
🚀 Tx sent: https://basescan.org/tx/0x303aadb75c02741ecd6f2ca97f37681dfb0d943e4fe83ec67055e067ad53d5e1


/tmp/ipykernel_330616/377798206.py:37: DeprecationWarning: The attribute rawTransaction on SignedTransaction is deprecated in favor of raw_transaction
  tx_hash = w3.eth.send_raw_transaction(signed_tx.rawTransaction)


In [33]:
def get_curation_listings(subgraph_url, curation_id, active_only=True):
    query = """
    {
      curationStorefront(id: "%s") {
        listings(where: {active: %s}) {
          id
          listingId
          storefront
          tokenId
          active
          createdAt
          lastUpdatedAt
          price
          paymentToken
          affiliateFee
          tokenURI
          erc1155Token
        }
      }
    }
    """ % (str(curation_id), str(active_only).lower())
    
    response = requests.post(subgraph_url, json={"query": query})
    response.raise_for_status()
    result = response.json()
    
    if "errors" in result:
        raise Exception(f"GraphQL Error: {result['errors']}")
    
    # Extract the listings from the curation
    curation = result["data"]["curationStorefront"]
    if curation:
        listings = curation["listings"]
        print(f"Found {len(listings)} listings for curation {curation_id}")
        return listings
    else:
        print(f"Curation {curation_id} not found")
        return []

listings = get_curation_listings(SUBGRAPH_URL, curation_id)
    
# Print sample listing
if listings:
  print("Sample listing:", json.dumps(listings[0], indent=2))


Found 1 listings for curation 8
Sample listing: {
  "id": "8-2",
  "listingId": "2",
  "storefront": "0x472375ba05fdc78caf9dd9d90a317c82504db1d9",
  "tokenId": "2",
  "active": true,
  "createdAt": "1743620863",
  "lastUpdatedAt": null,
  "price": "1000",
  "paymentToken": "0x0000000000000000000000000000000000000000",
  "affiliateFee": 1500,
  "tokenURI": "data:application/json;base64,eyJuYW1lIjogIlNvbGlkIEdvbGQgV2Fzc2llIFNjdWxwdHVyZSIsImRlc2NyaXB0aW9uIjogIlRoaXMgd2Fzc2llIHNjdWxwdHVyZSBpcyBtYWRlIGZyb20gMSBrZyBvZiBzb2xpZCBnb2xkIiwiaW1hZ2UiOiAiaHR0cHM6Ly9pcGZzLmlvL2lwZnMvYmFmeWJlaWNrcHc2ZTZtZG5pd2trM3RrcnVnc241aDVqcm9qNmdrZXEzM3hzNTVtc20zYWFzenU0aGUiLCJhdHRyaWJ1dGVzIjogW3sidHJhaXRfdHlwZSI6IlRlcm1zIG9mIFNlcnZpY2UiLCJ2YWx1ZSI6IlRlc3QgaXRlbSJ9LHsidHJhaXRfdHlwZSI6IlN1cHBsZW1lbnRhbCBJbWFnZXMiLCJ2YWx1ZSI6WyJodHRwczovL2lwZnMuaW8vaXBmcy9iYWZ5YmVpY2twdzZlNm1kbml3a2szdGtydWdzbjVoNWpyb2o2Z2tlcTMzeHM1NW1zbTNhYXN6dTRoZSJdfV19",
  "erc1155Token": "0xfae480c53a31ffef620fd5cec4fd6b90ac3a91ef"
}


just a cell for the encryption stuff

In [34]:
from eth_keys import keys
from eth_utils import keccak, to_bytes, to_hex
from Crypto.Cipher import AES
import os
import struct
from typing import Optional, Tuple, Union

class EthereumMessageEncryption:
    BLOCK_SIZES = [64, 128, 256, 512, 1024, 2048]  # Same block sizes as JS version
    
    def __init__(self, web3):
        self.web3 = web3
    
    def strip_hex_prefix(self, hex_str: str) -> str:
        """Remove 0x prefix from hex string if present."""
        return hex_str[2:] if hex_str.startswith('0x') else hex_str
    
    def order_public_keys(self, key1: str, key2: str) -> Tuple[str, str]:
        """Order two public keys lexicographically."""
        clean1 = self.strip_hex_prefix(key1)
        clean2 = self.strip_hex_prefix(key2)
        return (key1, key2) if clean1 < clean2 else (key2, key1)
    
    def derive_shared_secret(self, public_key1: str, public_key2: str) -> bytes:
        """Derive a shared secret from two public keys."""
        first_key, second_key = self.order_public_keys(public_key1, public_key2)
        combined = bytes.fromhex(self.strip_hex_prefix(first_key)) + bytes.fromhex(self.strip_hex_prefix(second_key))
        return keccak(combined)
    
    def get_target_block_size(self, message_length: int) -> int:
        """Get the appropriate block size for a message of given length."""
        for size in self.BLOCK_SIZES:
            if message_length <= size - 9:  # 9 bytes for length prefix
                return size
        raise ValueError(f"Message too long. Maximum size is {self.BLOCK_SIZES[-1] - 9} bytes")
    
    def pad_message(self, message_bytes: bytes) -> bytes:
        """Pad message according to the same scheme as JS version."""
        original_length = len(message_bytes)
        target_size = self.get_target_block_size(original_length)
        
        # Create length bytes (8 bytes, little endian)
        length_bytes = struct.pack('<Q', original_length)
        
        # Create padded message
        padded_message = bytearray(target_size)
        padded_message[0] = 8  # Length of length field
        padded_message[1:9] = length_bytes
        padded_message[9:9 + original_length] = message_bytes
        
        # Fill remaining space with random bytes
        padding = os.urandom(target_size - original_length - 9)
        padded_message[9 + original_length:] = padding
        
        return bytes(padded_message)
    
    def unpad_message(self, padded_message: bytes) -> bytes:
        """Unpad message according to the same scheme as JS version."""
        length_bytes_count = padded_message[0]
        length = struct.unpack('<Q', padded_message[1:1 + length_bytes_count])[0]
        return padded_message[9:9 + length]
    
    def create_message_hash(self, message: bytes, salt: bytes) -> str:
        """Create a hash of the message with salt."""
        salted_message = message + salt
        return to_hex(keccak(salted_message))
    
    def encrypt_message(self, message: str, recipient_public_key: str, include_verification: bool = False) -> dict:
        """
        Encrypt a message for a recipient using their public key.
        
        Args:
            message: The message to encrypt
            recipient_public_key: Recipient's public key
            include_verification: Whether to include verification hash
            
        Returns:
            dict: Encrypted message data including:
                - encryptedData
                - ephemeralPublicKey
                - iv
                - verificationHash (if verification enabled)
        """
        try:
            # Convert message to bytes
            message_bytes = message.encode('utf-8')
            
            padded_message: bytes
            verification_hash: Optional[str] = None
            salt: Optional[bytes] = None
            
            if include_verification:
                # Generate random salt and hash
                salt = os.urandom(32)
                verification_hash = self.create_message_hash(message_bytes, salt)
                
                # Combine message and salt
                message_with_salt = message_bytes + salt
                padded_message = self.pad_message(message_with_salt)
            else:
                padded_message = self.pad_message(message_bytes)
            
            # Generate ephemeral key pair
            ephemeral_private_key = keys.PrivateKey(os.urandom(32))
            ephemeral_public_key = ephemeral_private_key.public_key
            
            # Derive shared secret
            shared_secret = self.derive_shared_secret(
                recipient_public_key,
                to_hex(ephemeral_public_key.to_bytes())
            )
            
            # Generate IV for AES-GCM
            iv = os.urandom(12)
            
            # Use first 32 bytes of shared secret as key
            key = shared_secret[:32]
            
            # Encrypt using AES-GCM
            cipher = AES.new(key, AES.MODE_GCM, nonce=iv)
            encrypted_data, tag = cipher.encrypt_and_digest(padded_message)
            
            # Combine encrypted data and tag
            full_encrypted = encrypted_data + tag
            
            result = {
                "encryptedData": full_encrypted.hex(),
                "ephemeralPublicKey": ephemeral_public_key.to_hex(),
                "iv": iv.hex(),
                "scheme": "password"
            }
            
            if include_verification:
                result["verificationHash"] = verification_hash
                result["verificationEnabled"] = True
            
            return result
            
        except Exception as e:
            raise Exception(f"Failed to encrypt message: {str(e)}")

    def verify_message(self, message: str, salt: bytes, hash: str) -> bool:
        """Verify a decrypted message matches its hash."""
        message_bytes = message.encode('utf-8')
        computed_hash = self.create_message_hash(message_bytes, salt)
        return computed_hash == hash
def get_encryption_key(web3: Web3, storefront_address: str, storefront_abi) -> str:
    """
    Gets the encryption public key from a SimpleERC1155Storefront contract.
    
    Args:
        web3: Web3 instance connected to the appropriate network
        storefront_address: Address of the SimpleERC1155Storefront contract
        storefront_abi: The ABI for the storefront contract
    
    Returns:
        str: The encryption public key string
        
    Raises:
        ValueError: If the contract cannot be loaded or key cannot be retrieved
    """
    try:
        storefront_contract = web3.eth.contract(
            address=web3.to_checksum_address(storefront_address),
            abi=storefront_abi
        )
        encryption_key = storefront_contract.functions.encryptionPublicKey().call()
        
        if not encryption_key:
            raise ValueError("No encryption key set in storefront contract")
            
        return encryption_key
        
    except Exception as e:
        raise ValueError(f"Error getting encryption key: {str(e)}")    


In [35]:
from eth_abi import encode  

STOREFRONT_ABI = load_abi_from_file('AffiliateERC1155Storefront_ABI.json')
SEAPORT_ABI = load_abi_from_file('seaport_abi.json')
def purchase_curated_listing(curation_listings, buyer_private_key, storefront_abi, seaport_abi):
    """
    Purchase the first listing from a curation
    
    Args:
        curation_listings (list): List of curated listings from the subgraph
        buyer_private_key (str): Private key of the buyer
        storefront_abi (list): ABI for the storefront contract
        seaport_abi (list): ABI for the Seaport contract
        
    Returns:
        dict: Transaction receipt if successful, None otherwise
    """
    if not curation_listings or len(curation_listings) == 0:
        print("No listings found")
        return None
    
    # Get the first listing
    listing = curation_listings[0]
    print(f"Attempting to purchase listing: {listing['id']}")
    
    # Set up the buyer account
    buyer_account = Account.from_key(buyer_private_key)
    buyer_address = buyer_account.address
    print(f"Buyer address: {buyer_address}")
    
    # Extract necessary information from the listing
    storefront_address = Web3.to_checksum_address(listing['storefront'])
    token_id = int(listing['tokenId'])
    erc1155_address = Web3.to_checksum_address(listing['erc1155Token'])
    payment_token = Web3.to_checksum_address(listing['paymentToken'])
    price = int(listing['price'])
    
    print(f"Storefront: {storefront_address}")
    print(f"Token ID: {token_id}")
    print(f"ERC1155 Token: {erc1155_address}")
    print(f"Payment Token: {payment_token}")
    print(f"Price: {price}")
    
    try:
        # Get the storefront contract
        storefront_contract = w3.eth.contract(address=storefront_address, abi=storefront_abi)
        
        # If this is a curated listing, we want to use the curation's payment address as affiliate
        # Get the curation ID from the listing ID (format: "curationId-listingId")
        curation_id = listing['id'].split('-')[0]
        
        # Query the curation to get the payment address
        query = """
        {
          curationStorefront(id: "%s") {
            paymentAddress
          }
        }
        """ % curation_id
        
        subgraph_url = "https://api.studio.thegraph.com/query/90920/ump-affiliate-review-curation/version/latest"
        response = requests.post(subgraph_url, json={"query": query})
        result = response.json()
        
        if "errors" in result:
            raise Exception(f"GraphQL Error: {result['errors']}")
        
        curation = result["data"]["curationStorefront"]
        affiliate_address = curation["paymentAddress"]
        print(f"Using affiliate address from curation: {affiliate_address}")
        
        # Use your existing preview_order function to get the proper offer and consideration
        # We need to recreate it here since we don't have direct access to it
        
        # Create the input for previewOrder
        offer = [{
            'itemType': 3,  # ERC1155
            'token': erc1155_address,
            'identifier': token_id,
            'amount': 1  # Assuming we're always buying 1 token
        }]
        
        consideration = [{
            'itemType': 0 if payment_token == "0x0000000000000000000000000000000000000000" else 1,  # NATIVE or ERC20
            'token': payment_token,
            'identifier': 0,
            'amount': price
        }]
        
        print("Offer:", offer)
        print("Consideration:", consideration)
        
        # Call previewOrder
        preview_result = storefront_contract.functions.previewOrder(
            buyer_address,
            buyer_address,
            offer,
            consideration,
            "0x"  # Empty bytes for unused parameter
        ).call()
        
        print("Preview result:", preview_result)
        
        # The preview_result contains the correctly formatted offer and consideration
        offer_items = preview_result[0]
        consideration_items = preview_result[1]
        
        print("Offer items:", offer_items)
        print("Consideration items:", consideration_items)
        
        # Adjust the offer and consideration arrays for the required format
        # Just like in your working example
        adjusted_offer = [(item[0], item[1], item[2], item[3], item[3]) for item in offer_items]
        adjusted_consideration = [(item[0], item[1], item[2], item[3], item[3], item[4]) for item in consideration_items]
        
        print("Adjusted offer:", adjusted_offer)
        print("Adjusted consideration:", adjusted_consideration)
        
        # Get the encrypted message components (using the same test message as in your code)
        encrypted_message = {
            "encryptedData": bytes.fromhex("b31bf3a238ffcdc022697f3599f16211c23994b50dd90ac3abdb74a32140741003defa93bb064827aa165b8972328214b40440375b754a4a85bb6ffdd9fce13e36c5915ff5b5f8af8e2bfb6e90eb8815"),
            "ephemeralPublicKey": bytes.fromhex("046901a52162ee78d8f14c43c1e29c228211fb416e73d212a1b9608ee5d97d21d3440782ddbc07b88985d2f54c9b055876b110a5c8c6b98f5ff7be640397f2032e"),
            "iv": bytes.fromhex("882b87d42312fa211e1bc9df"),
            "verificationHash": b''  # Empty bytes for no verification hash
        }
        
        # Encode the affiliate address separately
        encoded_affiliate = encode(['address'], [Web3.to_checksum_address(affiliate_address)])
        
        # Encode the message components
        encoded_message = encode(
            ['bytes', 'bytes', 'bytes', 'bytes'],  # Separate types, not a tuple
            [
                encrypted_message['encryptedData'],
                encrypted_message['ephemeralPublicKey'],
                encrypted_message['iv'],
                encrypted_message['verificationHash']
            ]
        )
        
        # Combine the encoded parts
        context_data = encoded_affiliate + encoded_message
        
        # Get the Seaport address
        seaport_address = storefront_contract.functions.SEAPORT().call()
        seaport_contract = w3.eth.contract(address=seaport_address, abi=seaport_abi)
        
        # Create order parameters
        order_parameters = (
            storefront_address,
            "0x0000000000000000000000000000000000000000",
            adjusted_offer,
            adjusted_consideration,
            4,  # CONTRACT order type
            0,
            2**256 - 1,
            "0x0000000000000000000000000000000000000000000000000000000000000000",
            0,
            "0x0000000000000000000000000000000000000000000000000000000000000000",
            len(adjusted_consideration)
        )
        
        # Create advanced order
        advanced_order = (
            order_parameters,
            1,  # numerator
            1,  # denominator
            Web3.to_bytes(hexstr="0x"),  # signature
            context_data
        )
        
        # Check if we need to send ETH with the transaction
        is_native_payment = payment_token == '0x0000000000000000000000000000000000000000'
        native_value = price if is_native_payment else 0
        
        print("Context data hexstring:", context_data.hex())
        print("Offer:", adjusted_offer)
        print("Consideration:", adjusted_consideration)
        
        # Build and send the transaction
        tx = seaport_contract.functions.fulfillAdvancedOrder(
            advanced_order,
            [],  # criteriaResolvers
            "0x0000000000000000000000000000000000000000000000000000000000000000",  # fulfillerConduitKey
            buyer_address
        ).build_transaction({
            'from': buyer_address,
            'gas': 3000000,
            'gasPrice': w3.eth.gas_price,
            'nonce': w3.eth.get_transaction_count(buyer_address),
            'value': native_value
        })
        
        print("Transaction built, signing...")
        signed_tx = buyer_account.sign_transaction(tx)
        tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
        print(f"Transaction sent: {tx_hash.hex()}")
        
        tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Order fulfillment successful. Hash: {tx_receipt.transactionHash.hex()}")
        return tx_receipt
        
    except Exception as e:
        print(f"Error purchasing listing: {str(e)}")
        import traceback
        traceback.print_exc()
        return None
# Get the curation listings
SUBGRAPH_URL = "https://api.studio.thegraph.com/query/90920/ump-affiliate-review-curation/version/latest"
curation_id = 8
listings = get_curation_listings(SUBGRAPH_URL, curation_id)

# Purchase the first listing
if listings:
    BUYER_PRIVATE_KEY = os.getenv('BUYER_PRIVATE_KEY')
    receipt = purchase_curated_listing(listings, BUYER_PRIVATE_KEY, STOREFRONT_ABI, SEAPORT_ABI)
    if receipt:
        print("Purchase successful!")
    else:
        print("Purchase failed")
else:
    print("No listings found to purchase")
 
#NOTE: Settle transaction example https://basescan.org/tx/0x8495477694e4d7728054233faa8f40ede72aecf47a72b8d1cdb220e20f77a231

Found 1 listings for curation 8
Attempting to purchase listing: 8-2
Buyer address: 0xE360EE830BB3dF8Ab17C277F1802a2e861850b9B
Storefront: 0x472375ba05fdC78cAF9DD9d90a317c82504dB1D9
Token ID: 2
ERC1155 Token: 0xFaE480c53A31FfEf620fd5CEc4fd6b90Ac3a91eF
Payment Token: 0x0000000000000000000000000000000000000000
Price: 1000
Using affiliate address from curation: 0x9f4640d04371ff6b7886ade5323746388107723a
Offer: [{'itemType': 3, 'token': '0xFaE480c53A31FfEf620fd5CEc4fd6b90Ac3a91eF', 'identifier': 2, 'amount': 1}]
Consideration: [{'itemType': 0, 'token': '0x0000000000000000000000000000000000000000', 'identifier': 0, 'amount': 1000}]
Preview result: [[(3, '0xFaE480c53A31FfEf620fd5CEc4fd6b90Ac3a91eF', 2, 1)], [(0, '0x0000000000000000000000000000000000000000', 0, 1000, '0x9F20e758f777b2E17683Fe272a0100072547F516')]]
Offer items: [(3, '0xFaE480c53A31FfEf620fd5CEc4fd6b90Ac3a91eF', 2, 1)]
Consideration items: [(0, '0x0000000000000000000000000000000000000000', 0, 1000, '0x9F20e758f777b2E17683Fe272a0

test changing the payment address

In [38]:

def set_payment_address(curation_id, new_payment_address, account):
    """Update the payment address for a curation"""
    nonce = w3.eth.get_transaction_count(account.address)
    
    tx = curation_contract.functions.setPaymentAddress(
        curation_id,
        new_payment_address
    ).build_transaction({
        'from': account.address,
        'gas': 200000,
        'gasPrice': w3.eth.gas_price,
        'nonce': nonce,
    })
    
    signed_tx = account.sign_transaction(tx)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    print(f"Transaction sent: {tx_hash.hex()}")
    
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    print(f"Updated payment address for curation {curation_id} to {new_payment_address}")
    return tx_receipt

def test_payment_address(curation_id, new_payment_address):
    """Test updating the payment address"""
    # Get current details
    initial_details = get_curation_details(curation_id)
    
    # Set new payment address
    set_payment_address(curation_id, new_payment_address, owner_account)
    
    # Verify payment address was updated
    updated_details = get_curation_details(curation_id)
    assert updated_details[2].lower() == new_payment_address.lower(), "Payment address was not updated successfully"
    
    # Set payment address back to original
    set_payment_address(curation_id, initial_details[2], owner_account)
test_payment_address(curation_id, curator_account.address)


Curation 8 Details:
Name: Test Curation Collection
Description: A test curation collection for managing affiliate storefronts
Payment Address: 0x9f4640d04371ff6b7886ade5323746388107723a
Owner: 0x9f4640d04371ff6b7886ade5323746388107723a
Total Listings: 0
Transaction sent: 0xa8601fc436e0862b3a402cbb9a8bb506f7990d252782b848183cf25857baeaab
Updated payment address for curation 8 to 0x988B8c3AD5971f79D1a461D17430AfC5f1002EaA

Curation 8 Details:
Name: Test Curation Collection
Description: A test curation collection for managing affiliate storefronts
Payment Address: 0x988B8c3AD5971f79D1a461D17430AfC5f1002EaA
Owner: 0x9f4640d04371ff6b7886ade5323746388107723a
Total Listings: 0
Transaction sent: 0x93e10120b4b8268b8788b3d69d746ec1b5089d1c1fdd6cc8b323e3612e7287e1
Updated payment address for curation 8 to 0x9f4640d04371ff6b7886ade5323746388107723a


In [44]:
SUBGRAPH_URL = "https://api.studio.thegraph.com/query/90920/ump-affiliate-reviews/version/latest"
def query_subgraph(query, variables=None):
    """
    Query the subgraph for data
    
    Args:
        query (str): GraphQL query string
        variables (dict, optional): Variables for the query
    
    Returns:
        dict: JSON response from the subgraph
    """
    request_json = {'query': query}
    
    # Add variables to the request if provided
    if variables:
        request_json['variables'] = variables
    
    response = requests.post(
        SUBGRAPH_URL,
        json=request_json
    )
    
    if response.status_code != 200:
        print(f"Query failed with status code: {response.status_code}")
        return None
        
    return response.json()

def get_active_storefronts():
    """
    Query subgraph for active storefronts with listings and review metrics
    """
    query = """
    query {
      storefronts(where: {ready: true}) {
        id
        storefrontAddress
        owner
        erc1155Token
        ready
        
        # Review statistics
        totalRating
        reviewCount
        
        # Include listings information
        listings(where: { active: true }) {
          id
          tokenId
          price
          paymentToken
          listingTime
          tokenURI
          affiliateFee
        }
        
        # Get orders with latest attestations
        orders(first: 100, orderBy: latestAttestationTimestamp, orderDirection: desc) {
          id
          buyer
          tokenId
          latestAttestationId
          saleAttestations(where: {isLatest: true}, first: 1) {
            id
            timestamp
            reviews {
              id
              reviewer
              reviewType
              overallRating
              qualityRating
              asDescribed
              reviewText
              attestationTxHash
              transactionHash
            }
          }
        }
      }
    }
    """
    
    result = query_subgraph(query)
    if not result or 'data' not in result:
        print("No data returned from query")
        return []
    
    storefronts = result['data']['storefronts']
    
    # Process review statistics for each storefront
    for sf in storefronts:
        # Calculate average rating if there are reviews
        sf['averageRating'] = float(sf['totalRating']) / float(sf['reviewCount']) if sf['reviewCount'] and int(sf['reviewCount']) > 0 else 0
        
        # Extract review metrics from orders
        total_reviews = 0
        as_described_count = 0
        positive_review_count = 0  # Reviews with 4+ stars
        
        # Process orders with attestations and reviews
        for order in sf['orders']:
            # Skip orders without attestations
            if not order['saleAttestations'] or len(order['saleAttestations']) == 0:
                continue
                
            # Process reviews for each attestation
            for attestation in order['saleAttestations']:
                for review in attestation['reviews']:
                    total_reviews += 1
                    
                    # Count "as described" reviews
                    if review['asDescribed']:
                        as_described_count += 1
                    
                    # Count positive reviews (4-5 stars)
                    if int(review['overallRating']) >= 4:
                        positive_review_count += 1
        
        # Add metrics to storefront object
        sf['totalReviews'] = total_reviews
        sf['asDescribedCount'] = as_described_count
        sf['asDescribedPercentage'] = (as_described_count / total_reviews * 100) if total_reviews > 0 else 0
        sf['positiveReviewCount'] = positive_review_count
        sf['positiveReviewPercentage'] = (positive_review_count / total_reviews * 100) if total_reviews > 0 else 0
    
    # Filter to only include storefronts with active listings
    active_storefronts = [sf for sf in storefronts if sf['listings']]
    print(f"Found {len(active_storefronts)} active storefronts with listings")
    
    return active_storefronts

# Get active storefronts
storefronts = get_active_storefronts()
print(json.dumps(storefronts, indent=2))

Found 11 active storefronts with listings
[
  {
    "id": "0x0b468e5e8c8df2d875ea83c564ed509f2d8281c1",
    "storefrontAddress": "0x0b468e5e8c8df2d875ea83c564ed509f2d8281c1",
    "owner": "0xefd5d2c7a831981b5e7ebf12226b08ab3097ece0",
    "erc1155Token": "0x6c62649755c4d9af00c36ef33d0e5305627991a1",
    "ready": true,
    "totalRating": "0",
    "reviewCount": "0",
    "listings": [
      {
        "id": "0x0b468e5e8c8df2d875ea83c564ed509f2d8281c1-1",
        "tokenId": "1",
        "price": "1000",
        "paymentToken": "0x0000000000000000000000000000000000000000",
        "listingTime": "1737902341",
        "tokenURI": "data:application/json;base64,eyJuYW1lIjogIldhc3NpZSdzIEdob3N0IFBlcHBlciBIb3QgU2F1Y2UiLCJkZXNjcmlwdGlvbiI6ICJUZXN0JTIwaXRlbSIsImltYWdlIjogImlwZnM6Ly9iYWZ5YmVpZnRzYmJqYWQ0NjdlcGwzYXlqcGE2ejVqZXVub3Bia2NwN2Z3MnprNzVreWtsMzZzYm9hdSIsImF0dHJpYnV0ZXMiOiBbeyJ0cmFpdF90eXBlIjoiVGVybXMgb2YgU2VydmljZSIsInZhbHVlIjoidGVzdCJ9LHsidHJhaXRfdHlwZSI6IlN1cHBsZW1lbnRhbCBJbWFnZXMiLCJ2YWx

In [25]:
def check_specific_storefront(storefront_address):
    """
    Query subgraph for a specific storefront by address
    """
    query = """
    query($address: ID!) {
      storefront(id: $address) {
        id
        storefrontAddress
        owner
        erc1155Token
        ready
        isAffiliateEnabled
        listings {
          id
          tokenId
          price
          paymentToken
          active
          listingTime
        }
      }
    }
    """
    
    variables = {
        "address": storefront_address.lower()  # Graph IDs are typically lowercase
    }
    
    result = query_subgraph(query, variables)
    if not result or 'data' not in result or not result['data']['storefront']:
        print(f"Storefront {storefront_address} not found in subgraph")
        return None
    
    storefront = result['data']['storefront']
    active_listings = [l for l in storefront['listings'] if l['active']]
    print(f"Storefront {storefront_address}:")
    print(f"- Ready: {storefront['ready']}")
    print(f"- Affiliate enabled: {storefront['isAffiliateEnabled']}")
    print(f"- Total listings: {len(storefront['listings'])}")
    print(f"- Active listings: {len(active_listings)}")
    
    return storefront

# Check the specific storefront
storefront = check_specific_storefront("0x472375ba05fdC78cAF9DD9d90a317c82504dB1D9")

Storefront 0x472375ba05fdC78cAF9DD9d90a317c82504dB1D9 not found in subgraph


In [47]:
def check_specific_storefront(storefront_address):
    """
    Query subgraph for a specific storefront by address
    """
    query = """
    query GetStorefront($address: ID!) {
      storefront(id: $address) {
        id
        storefrontAddress
        owner
        erc1155Token
        ready
        isAffiliateEnabled
        listings {
          id
          tokenId
          price
          paymentToken
          active
          listingTime
        }
      }
    }
    """
    
    variables = {
        "address": storefront_address.lower()  # Graph IDs are typically lowercase
    }
    
    result = query_subgraph(query, variables)
    if not result or 'data' not in result or not result['data']['storefront']:
        print(f"Storefront {storefront_address} not found in subgraph")
        return None
    
    storefront = result['data']['storefront']
    active_listings = [l for l in storefront['listings'] if l['active']]
    print(f"Storefront {storefront_address}:")
    print(f"- Ready: {storefront['ready']}")
    print(f"- Affiliate enabled: {storefront['isAffiliateEnabled']}")
    print(f"- Total listings: {len(storefront['listings'])}")
    print(f"- Active listings: {len(active_listings)}")
    
    return storefront

def check_all_storefronts():
    """
    Query all storefronts regardless of ready status
    """
    query = """
    query {
      storefronts {
        id
        storefrontAddress
        owner
        ready
        isAffiliateEnabled
        listings {
          id
          active
        }
      }
    }
    """
    
    result = query_subgraph(query)
    if not result or 'data' not in result:
        print("No data returned from query")
        return []
    
    storefronts = result['data']['storefronts']
    print(f"Found {len(storefronts)} total storefronts")
    
    # Count by different criteria
    ready_count = len([sf for sf in storefronts if sf['ready']])
    affiliate_count = len([sf for sf in storefronts if sf['isAffiliateEnabled']])
    with_listings = len([sf for sf in storefronts if sf['listings']])
    with_active_listings = len([sf for sf in storefronts if any(l['active'] for l in sf['listings'])])
    
    print(f"- Ready storefronts: {ready_count}")
    print(f"- Affiliate-enabled storefronts: {affiliate_count}")
    print(f"- Storefronts with any listings: {with_listings}")
    print(f"- Storefronts with active listings: {with_active_listings}")
    
    # Check if our specific storefront is in the list
    target_address = "0x472375ba05fdC78cAF9DD9d90a317c82504dB1D9".lower()
    target_storefront = next((sf for sf in storefronts if sf['id'].lower() == target_address), None)
    
    if target_storefront:
        print(f"\nTarget storefront found:")
        print(f"- Ready: {target_storefront['ready']}")
        print(f"- Affiliate enabled: {target_storefront['isAffiliateEnabled']}")
        print(f"- Has listings: {len(target_storefront['listings']) > 0}")
        print(f"- Has active listings: {any(l['active'] for l in target_storefront['listings'])}")
    else:
        print(f"\nTarget storefront NOT found in subgraph")
    
    return storefronts

def get_active_storefronts():
    """
    Query subgraph for active storefronts with listings and review metrics
    """
    query = """
    query {
      storefronts(where: {ready: true}) {
        id
        storefrontAddress
        owner
        erc1155Token
        ready
        isAffiliateEnabled
        
        # Review statistics
        totalRating
        reviewCount
        
        # Include listings information
        listings(where: { active: true }) {
          id
          tokenId
          price
          paymentToken
          listingTime
          tokenURI
          affiliateFee
        }
        
        # Get orders with latest attestations
        orders(first: 100, orderBy: latestAttestationTimestamp, orderDirection: desc) {
          id
          buyer
          tokenId
          latestAttestationId
          saleAttestations(where: {isLatest: true}, first: 1) {
            id
            timestamp
            reviews {
              id
              reviewer
              reviewType
              overallRating
              qualityRating
              asDescribed
              reviewText
              attestationTxHash
              transactionHash
            }
          }
        }
      }
    }
    """
    
    result = query_subgraph(query)
    if not result or 'data' not in result:
        print("No data returned from query")
        return []
    
    storefronts = result['data']['storefronts']
    
    # Process review statistics for each storefront
    for sf in storefronts:
        # Calculate average rating if there are reviews
        sf['averageRating'] = float(sf['totalRating']) / float(sf['reviewCount']) if sf['reviewCount'] and int(sf['reviewCount']) > 0 else 0
        
        # Extract review metrics from orders
        total_reviews = 0
        as_described_count = 0
        positive_review_count = 0  # Reviews with 4+ stars
        
        # Process orders with attestations and reviews
        for order in sf['orders']:
            # Skip orders without attestations
            if not order['saleAttestations'] or len(order['saleAttestations']) == 0:
                continue
                
            # Process reviews for each attestation
            for attestation in order['saleAttestations']:
                for review in attestation['reviews']:
                    total_reviews += 1
                    
                    # Count "as described" reviews
                    if review['asDescribed']:
                        as_described_count += 1
                    
                    # Count positive reviews (4-5 stars)
                    if int(review['overallRating']) >= 4:
                        positive_review_count += 1
        
        # Add metrics to storefront object
        sf['totalReviews'] = total_reviews
        sf['asDescribedCount'] = as_described_count
        sf['asDescribedPercentage'] = (as_described_count / total_reviews * 100) if total_reviews > 0 else 0
        sf['positiveReviewCount'] = positive_review_count
        sf['positiveReviewPercentage'] = (positive_review_count / total_reviews * 100) if total_reviews > 0 else 0
    
    # Filter to only include storefronts with active listings
    active_storefronts = [sf for sf in storefronts if sf['listings']]
    print(f"Found {len(active_storefronts)} active storefronts with listings")
    
    return active_storefronts

# Now modify the original function to include storefronts regardless of ready status
def get_all_storefronts_with_listings():
    """
    Query all storefronts with listings, including those that aren't ready
    """
    query = """
    query {
      storefronts {
        id
        storefrontAddress
        owner
        erc1155Token
        ready
        isAffiliateEnabled
        listings(where: { active: true }) {
          id
          tokenId
          price
          paymentToken
          listingTime
          tokenURI
          affiliateFee
        }
      }
    }
    """
    
    result = query_subgraph(query)
    if not result or 'data' not in result:
        print("No data returned from query")
        return []
    
    storefronts = result['data']['storefronts']
    storefronts_with_listings = [sf for sf in storefronts if sf['listings']]
    print(f"Found {len(storefronts_with_listings)} storefronts with active listings (regardless of ready status)")
    
    # Check if the target storefront is in this list
    target_address = "0x472375ba05fdC78cAF9DD9d90a317c82504dB1D9".lower()
    target_storefront = next((sf for sf in storefronts_with_listings if sf['id'].lower() == target_address), None)
    
    if target_storefront:
        print(f"\nTarget storefront has active listings but is ready={target_storefront['ready']}")
    else:
        print(f"\nTarget storefront does not have active listings in the subgraph")
    
    return storefronts_with_listings
# First, check if the specific storefront exists
storefront = check_specific_storefront("0x472375ba05fdC78cAF9DD9d90a317c82504dB1D9")

# If that doesn't work, check all storefronts
all_storefronts = check_all_storefronts()

# Check all storefronts with listings regardless of ready status
all_with_listings = get_all_storefronts_with_listings()

Storefront 0x472375ba05fdC78cAF9DD9d90a317c82504dB1D9:
- Ready: False
- Affiliate enabled: True
- Total listings: 1
- Active listings: 1
Found 30 total storefronts
- Ready storefronts: 11
- Affiliate-enabled storefronts: 9
- Storefronts with any listings: 18
- Storefronts with active listings: 17

Target storefront found:
- Ready: False
- Affiliate enabled: True
- Has listings: True
- Has active listings: True
Found 17 storefronts with active listings (regardless of ready status)

Target storefront has active listings but is ready=False


In [39]:


def curate_listing(curation_id, storefront_address, token_id, account):
    """Add a listing to a curation"""
    nonce = w3.eth.get_transaction_count(account.address)
    
    tx = curation_contract.functions.curateListing(
        curation_id,
        Web3.to_checksum_address(storefront_address),
        token_id
    ).build_transaction({
        'from': account.address,
        'gas': 300000,
        'gasPrice': w3.eth.gas_price,
        'nonce': nonce,
    })
    
    signed_tx = account.sign_transaction(tx)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    print(f"Transaction sent: {tx_hash.hex()}")
    
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    
    if tx_receipt.status == 0:
        print(f"Transaction failed. The storefront at {storefront_address} may not be compatible with the curation contract.")
        return tx_receipt, None
    
    # Wait a bit to ensure the transaction is properly indexed
    time.sleep(2)
    
    # Get updated curation details to see if our listing was added
    details = curation_contract.functions.getCurationDetails(curation_id).call()
    total_listings = details[4]
    
    if total_listings > 0:
        # Try to get the most recent listing
        listing_id = total_listings
        print(f"Listing likely added as ID {listing_id}")
        return tx_receipt, listing_id
    else:
        print("No listings found after adding one. The transaction may have failed silently.")
        return tx_receipt, None
curate_listing(8, "0xEd265E2525d2821500Be786343EA633F564351F9", 2, owner_account)
time.sleep(5)
curate_listing(8, "0x472375ba05fdC78cAF9DD9d90a317c82504dB1D9", 2, owner_account)


Transaction sent: 0x145c17856bd3955438b38ececf097c8dd86a3be56025f17f090da7ce6e90c020
Listing likely added as ID 1
Transaction sent: 0x27d586001e0ec13c66a6452508895a463ebd2f1fe5d171d33ba00fe9fc385630
Listing likely added as ID 2


(AttributeDict({'blockHash': HexBytes('0x0ed43a6f9f8cdda75ad64bd8f83acf643654f7aa567ce8f61da818a087708073'),
  'blockNumber': 28415758,
  'contractAddress': None,
  'cumulativeGasUsed': 44309625,
  'effectiveGasPrice': 1155920,
  'from': '0x9f4640d04371ff6b7886ade5323746388107723a',
  'gasUsed': 113253,
  'l1BaseFeeScalar': '0x8dd',
  'l1BlobBaseFee': '0x43c',
  'l1BlobBaseFeeScalar': '0x101c12',
  'l1Fee': '0x8035e40d',
  'l1GasPrice': '0x23505daf',
  'l1GasUsed': '0x640',
  'logs': [AttributeDict({'address': '0xdFCA83ff7544Acb88B5D04A9101d58780243E0cb',
    'blockHash': HexBytes('0x0ed43a6f9f8cdda75ad64bd8f83acf643654f7aa567ce8f61da818a087708073'),
    'blockNumber': 28415758,
    'data': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000002'),
    'logIndex': 507,
    'removed': False,
    'topics': [HexBytes('0x77f721762106d67c19791d1ec46ac5a8abf43cd46ed517bbcf213ecd1fc9a180'),
     HexBytes('0x0000000000000000000000000000000000000000000000000000000000000008'

In [40]:
def update_listing(curation_id, listing_id, active, account):
    """Update a listing's active status"""
    nonce = w3.eth.get_transaction_count(account.address)
    
    tx = curation_contract.functions.updateListing(
        curation_id,
        listing_id,
        active
    ).build_transaction({
        'from': account.address,
        'gas': 200000,
        'gasPrice': w3.eth.gas_price,
        'nonce': nonce,
    })
    
    signed_tx = account.sign_transaction(tx)
    tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
    print(f"Transaction sent: {tx_hash.hex()}")
    
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    status = "active" if active else "inactive"
    print(f"Updated listing {listing_id} in curation {curation_id} to {status}")
    return tx_receipt

#deactivate listing #1
update_listing(8, 1, False, owner_account)


Transaction sent: 0xb2bbc6116f53fad106e54fd802dd51aa44bf1b3d834f09413c6d6f32900f5a8d
Updated listing 1 in curation 8 to inactive


AttributeDict({'blockHash': HexBytes('0x94b381fdd35aefb94c50ad86398e40583ffa09701d784da6c12cab6872f5582a'),
 'blockNumber': 28415794,
 'contractAddress': None,
 'cumulativeGasUsed': 37080937,
 'effectiveGasPrice': 1127277,
 'from': '0x9f4640d04371ff6b7886ade5323746388107723a',
 'gasUsed': 28830,
 'l1BaseFeeScalar': '0x8dd',
 'l1BlobBaseFee': '0x412',
 'l1BlobBaseFeeScalar': '0x101c12',
 'l1Fee': '0x79f1662e',
 'l1GasPrice': '0x21966cb0',
 'l1GasUsed': '0x640',
 'logs': [AttributeDict({'address': '0xdFCA83ff7544Acb88B5D04A9101d58780243E0cb',
   'blockHash': HexBytes('0x94b381fdd35aefb94c50ad86398e40583ffa09701d784da6c12cab6872f5582a'),
   'blockNumber': 28415794,
   'data': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
   'logIndex': 309,
   'removed': False,
   'topics': [HexBytes('0x31aceb5c222c3c2acf0bfdd74237844684c71194bdb3442f13492d1bfceb950b'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000008'),
    HexBytes('0x000

In [41]:
#get listing details from curation
get_curated_listing(8, 1)


Listing 1 in Curation 8:
Storefront Address: 0xEd265E2525d2821500Be786343EA633F564351F9
Token ID: 2
Active: False
Price: 1000
Payment Token: 0x0000000000000000000000000000000000000000
Affiliate Fee: 15.0%
ERC1155 Token: 0x7CC71776aDC93E65554bDEEC5c344d701E0D43C7


['0xEd265E2525d2821500Be786343EA633F564351F9',
 2,
 False,
 1000,
 '0x0000000000000000000000000000000000000000',
 1500,
 '0x7CC71776aDC93E65554bDEEC5c344d701E0D43C7']